# German real estate listings over time

## 0) Imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import glob

%load_ext nb_black
%matplotlib inline

plt.rcParams["figure.figsize"] = (15, 10)
pd.set_option("display.precision", 2)

<IPython.core.display.Javascript object>

## 1) Data Engineering

### 1.0) Helper functions:

In [2]:
def load_df(filepath):
    df = pd.read_csv(filepath, sep=";", low_memory=False)
    df.columns = [
        x.replace("obj_", "").replace("ga_", "").replace("geo_", "") for x in df.columns
    ]
    df = df.drop_duplicates(subset="scoutId")
    df = df[df.columns[~df.isnull().any()]]
    return df


def create_combined_df(date, rent_df, sale_df, group):
    rent_df["rent_m2"] = rent_df["baseRent"] / rent_df["livingSpace"]
    sale_df["price_m2"] = sale_df["purchasePrice"] / sale_df["livingSpace"]

    g = rent_df.groupby("zipCode")["rent_m2"].median()
    g.name = "rent_m2_zipCode"
    sale_df = sale_df.join(g, on="zipCode").copy()
    sale_df["yearly_ROI"] = sale_df["rent_m2_zipCode"] * 12 / sale_df["price_m2"]

    combined_df = pd.concat(
        [
            rent_df.groupby(group)["rent_m2"].median(),
            rent_df.groupby(group)["rent_m2"].size(),
            sale_df.groupby(group)["price_m2"].median(),
            sale_df.groupby(group)["price_m2"].size(),
            sale_df.groupby(group)["yearly_ROI"].median(),
        ],
        axis=1,
        keys=[
            "median_rent_m2 " + date,
            "num_rent " + date,
            "median_price_m2 " + date,
            "num_sale " + date,
            "median_yearly_ROI " + date,
        ],
    ).copy()
    return combined_df


def compute_change(df_column, startdate, enddate):
    return time_df[df_column + enddate] / time_df[df_column + startdate]

<IPython.core.display.Javascript object>

### 1.1) Data processing:

#### find dates:

In [3]:
dates = sorted(
    [
        s[s.find("mieten") + len("mieten") : s.rfind(".csv")]
        for s in glob.glob("../data/mieten*")
    ]
)

<IPython.core.display.Javascript object>

#### create time series dataframe:

In [4]:
group = "zipCode"
time_df = pd.DataFrame()

for date in dates:
    rent_filepath = "../data/mieten" + date + ".csv"
    sale_filepath = "../data/kaufen" + date + ".csv"
    rent_df = load_df(rent_filepath)
    sale_df = load_df(sale_filepath)
    combined_df = create_combined_df(date, rent_df, sale_df, group)
    time_df = pd.concat([time_df, combined_df], axis=1)

time_df = time_df.replace([np.inf, -np.inf], np.nan)
time_df = time_df.dropna()

<IPython.core.display.Javascript object>

#### add columns for change:

In [5]:
startdate = dates[0]
enddate = dates[-1]

time_df["median_rent_m2 change"] = compute_change("median_rent_m2 ", startdate, enddate)
time_df["num_rent change"] = compute_change("num_rent ", startdate, enddate)
time_df["median_price_m2 change"] = compute_change(
    "median_price_m2 ", startdate, enddate
)
time_df["num_sale change"] = compute_change("num_sale ", startdate, enddate)
time_df["median_yearly_ROI change"] = compute_change(
    "median_yearly_ROI ", startdate, enddate
)

move_cols = [
    "median_rent_m2 change",
    "num_rent change",
    "median_price_m2 change",
    "num_sale change",
    "median_yearly_ROI change",
]
time_df = time_df[move_cols + [col for col in time_df.columns if col not in move_cols]]

<IPython.core.display.Javascript object>

## 2) Results

In [8]:
min_num = 1
startdate = dates[0]
enddate = dates[-1]
time_ord = "median_yearly_ROI change"

time_filt = (
    (time_df["num_rent " + startdate] > min_num)
    & (time_df["num_rent " + startdate] > min_num)
    & (time_df["num_sale " + enddate] > min_num)
    & (time_df["num_sale " + enddate] > min_num)
)

time_df[time_filt].sort_values(time_ord, ascending=False)

,median_rent_m2 change,num_rent change,median_price_m2 change,num_sale change,median_yearly_ROI change,median_rent_m2 2020-04-04,num_rent 2020-04-04,median_price_m2 2020-04-04,num_sale 2020-04-04,median_yearly_ROI 2020-04-04,...,median_rent_m2 2020-04-16,num_rent 2020-04-16,median_price_m2 2020-04-16,num_sale 2020-04-16,median_yearly_ROI 2020-04-16,median_rent_m2 2020-04-19,num_rent 2020-04-19,median_price_m2 2020-04-19,num_sale 2020-04-19,median_yearly_ROI 2020-04-19
zipCode,,,,,,,,,,,,,,,,,,,,,
93449,1.08,1.00,0.39,1.67,2.79,6.41,4.0,1000.00,3.0,0.08,...,6.90,4.0,386.36,5.0,0.21,6.90,4.0,386.36,5.0,0.21
17039,1.00,1.00,0.62,2.00,2.64,6.71,2.0,2125.67,1.0,0.04,...,6.71,2.0,1311.66,2.0,0.10,6.71,2.0,1311.66,2.0,0.10
73650,2.21,0.50,0.92,1.00,2.39,11.08,4.0,3566.64,6.0,0.04,...,24.43,2.0,3267.05,6.0,0.09,24.43,2.0,3267.05,6.0,0.09
9120,0.97,1.04,0.44,0.58,2.20,5.23,162.0,3043.79,12.0,0.02,...,5.10,168.0,1348.31,7.0,0.05,5.10,169.0,1348.31,7.0,0.05
35614,1.06,0.92,0.66,2.00,2.18,8.50,13.0,2954.55,1.0,0.03,...,8.99,12.0,1950.76,2.0,0.08,8.99,12.0,1950.76,2.0,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67722,0.90,1.00,2.04,3.00,0.44,6.94,3.0,1200.00,1.0,0.07,...,6.27,3.0,2450.00,3.0,0.03,6.27,3.0,2450.00,3.0,0.03
13583,0.57,1.67,1.30,1.60,0.44,13.13,3.0,2444.44,5.0,0.06,...,8.82,5.0,3251.85,9.0,0.03,7.43,5.0,3165.89,8.0,0.03
52223,0.98,1.00,2.54,1.00,0.39,6.73,14.0,621.21,5.0,0.13,...,6.73,13.0,727.27,7.0,0.11,6.62,14.0,1578.57,5.0,0.05


<IPython.core.display.Javascript object>